# Project : v2 - Customer discount specialist agent


In [1]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
import json

## Setup LLM

In [2]:
from dotenv import load_dotenv
import sys
import json

from langchain.prompts import PromptTemplate

# Load the file that contains the API keys - OPENAI_API_KEY
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

# setting path
sys.path.append('../')

from utils.create_chat_llm import create_gpt_chat_llm, create_cohere_chat_llm, create_anthropic_chat_llm

# Try with GPT
llm = create_gpt_chat_llm({"temperature":0.1, "model_kwargs":{"top_p":0.1}})

## 1. Setup prompt

https://smith.langchain.com/hub/hwchase17/react-json

In [3]:
# prompt_template = """
# You are an agent responsible for determining the discount on purchases made by customers on an e-commerce website. 

# Use the tools to determine the discount percentage for the customer.

# Customer is identified by their id, which will be provided to you. 
# Use the promo code: {promo_code}

# Final Answer: answer should be in valid JSON format {{loyalty_discount: percentage, promo_discount: percentage}}

# Begin!

# Customer ID: {customer_id}

# Thought:{agent_scratchpad}
# """

# prompt = PromptTemplate(template=prompt_template, input_variables=["promo_code", "customer_id", "agent_scratchpad"])

# system_message = """
# You are a sales agent responsible for determining the discount percentage on purchases made by customers on an e-commerce website. 
# You can learn about our discount policy by going over our company's discount policy document. 
# You MUST apply the rules described in the discount policy document to calculate the final discount.
# Use only the given tools and information provided by the customer to determine the discount percentage for the customer.
# No need to preamble, just provide the final discount calculations in the following valid JSON format:

# {{"customer_discount": final_discount_percentage as float, 
#   "loyalty_discount": loyalty_discount_prcentage as float, 
#   "promo_discount":promo_discount_percentage, 
#   "was_final_discount_adjusted": True if the discount was reduced as a result of maximum discount policy, otherwise False}}

# """

system_message = """
You are a sales agent responsible for calculating the discount percentage for customer purchases on our e-commerce website.

Your task is to determine the final discount using our company's discount policy. 
You can learn the discount policy by reviewing the document (version 1.0) to understand the rules for calculating discounts.

You MUST apply the rules described in the discount policy document to calculate the final customer discount.

Use only the discount policy, tools and information provided by the customer to determine the discount percentage.

Provide the final output in the following RFC8259 compliant JSON format, without any extra text or explanations:

{{
  "customer_discount": <final_discount_percentage as float>,
  "loyalty_points_balance": <loyalty points balance for the customer>,
  "loyalty_discount": <loyalty_discount_percentage as float>,
  "promo_discount": <promo_discount_percentage as float>,
  "was_final_discount_adjusted": <True or False>
}}

The json output generated by you will be parsed by application code so do not add any extraneous characters around it.

You should always think about what to do.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", "my customer_id is {customer_id} and I also have a promo code {promo_code}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# prompt = PromptTemplate(template=prompt_template, input_variables=["promo_code", "customer_id", "agent_scratchpad"])




## 2. Tools

In [4]:
@tool
def get_customer_loyalty_points_balance(customer_id):
    """
    Retrieves the loyalty points balance for a given customer.

    This function returns the loyalty points balance for customers with specific IDs.
    If the customer ID is not recognized, it returns a balance of 0.

    Args:
        customer_id (int): The unique identifier for the customer.

    Returns:
        int: The loyalty points balance for the customer.

    """
    if customer_id == 100:
        return 342
    elif customer_id == 200:
        return 599
    elif customer_id == 300:
        return 1001
    elif customer_id == 400:
        return 1785
    else:
        return 0       

In [5]:
@tool
def get_discount_policy_document(policy_version):
    """
    Retrieves the discount policy document for a specified policy version.

    This function returns the discount policy document associated with the given policy version. 
    You can use the discount poilcy to calaculate the discount percentage for a customer.
    
    Currently, the recognized policy version is "1.0". If an invalid policy version 
    is provided, the function returns a message indicating that the policy version is invalid.

    Args:
        policy_version (str): The version of the discount policy to retrieve.

    Returns:
        str: The discount policy for the specified version, or an error message 
             if the policy version is invalid.

    Example:
        >>> get_discount_policy("1.0")
        'if customer loyalty point balance is equal to 0 then the discount is 0% ...'

        >>> get_discount_policy("2.0")
        'Invalid policy version'
    """

    # discount_policy = """
    # There are two types of discounts.

    # 1. Loyalty discounts:
    # Customers earns loyalty points as they purchase items on our website. 
    # The customer's loyalty points balance is used for determining the discount percentage for the customer.
    # Higher the loyalty points balanace, more is the discount percentage.

    # 2. Promotional discount:
    # Time to time we give out promotional discount codes. 
    # The discount percentage depends on the promotion code that customer presents at the time of purchase.

    # Final discount calculations:
    # The final discount for the customer is caluculated by adding the loyalty discount percentage and promotional discount percentage.

    # Maximum discount policy:
    # The customer discount cannot exceed 25%.
    # If the final discount is greater than 25%, change it to 25% and mark the discount as adjusted.
    
    # """

    discount_policy = """
    There are two types of discounts:

    Loyalty Discounts:
    
    Customers earn loyalty points when they purchase items on our website. 
    The customer's loyalty points balance determines the loyalty discount percentage they receive. 
    The higher the loyalty points balance, the greater the discount percentage.
    
    Promotional Discounts:
    
    From time to time, we offer promotional discount codes. 
    The discount percentage depends on the promotion code presented by the customer at the time of purchase.
    
    Final Discount Calculations:
    
    The final discount for a customer is calculated by adding the loyalty discount percentage and the promotional discount percentage.
    
    Maximum Discount Policy:
    
    The customer discount cannot exceed 25%. If the final discount is greater than 25%, it is adjusted to 25%, and the discount is marked as adjusted.
    """
    
    if policy_version == "1.0":
        return discount_policy
    else:
        return "Invalid policy version"



In [6]:
@tool
def get_discount_pct_for_loyalty(loyalty_points_balance):
    """
    This function determines the discount percentage a customer is eligible for based on 
    their loyalty points balance. 

    Args:
        loyalty_points_balance (int): The customer's loyalty points balance.

    Returns:
        float: The discount percentage applicable to the customer.

    """
    if loyalty_points_balance >= 500 and loyalty_points_balance < 1000:
        return 0.05
    elif loyalty_points_balance >= 1000 and loyalty_points_balance < 1500:
        return 0.10
    elif loyalty_points_balance >= 1500:
        return 0.15
    else:
        return 0.0


In [7]:
@tool
def get_discount_pct_for_promotion_code(promo_code):
    """
    Retrieves the discount percentage associated with a given promotion code.

    This function checks the provided promotion code and returns the corresponding 
    discount percentage. 

    If the promotion code is not recognized, the function returns a discount of 0%.

    Args:
        promo_code (str): The promotion code to check.

    Returns:
        float: The discount percentage for the given promotion code.

    """
    if promo_code == "friends&family":
        return 0.10
    elif promo_code == "summer2024":
        return 0.15
    else:
        return 0


In [8]:
@tool
def add_pct_tool(pct1: float, pct2: float) -> float:
    """
    Adds two percentage values and returns the result.

    This function takes two floating-point numbers representing percentages and returns their sum. 
    
    Args:
        pct1 (float): The first percentage value.
        pct2 (float): The second percentage value.

    Returns:
        float: The sum of the two percentage values.

    Example:
        >>> add_pct_tool(0.10, 0.15)
        0.25

        >>> add_pct_tool(0.05, 0.10)
        0.15
    """
    return pct1 + pct2


## 3. Create the agent & executor
Comment the cell based on your desired agent

#### Use tool calling agent (doesn't use OpenAI function capability)

create_tool_calling_agent

In [9]:
tools = [get_customer_loyalty_points_balance, get_discount_policy_document, get_discount_pct_for_loyalty, get_discount_pct_for_promotion_code,add_pct_tool]

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

#### Use OpenAI tool capability

create_openai_tools_agent

In [10]:
# from langchain.agents import AgentExecutor, create_openai_tools_agent

# tools = [get_customer_loyalty_points_balance, get_discount_policy_document, get_discount_pct_for_loyalty, get_discount_pct_for_promotion_code,add_pct_tool]

# agent = create_openai_tools_agent(llm, tools, prompt)

# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

## 4. Test harness

In [11]:
# Sometimes ChatGPT wraps the JSON with ```json<JSON>```
# This function blindly replaces the undesired tags from the output
# https://community.openai.com/t/getting-response-data-as-a-fixed-consistent-json-response/28471/23
def  openai_chat_output_hack(output):
    str = output.replace("```json","")
    str = str.replace("```","")
    return str

In [12]:
def calculate_discount_without_LLM(customer_id, promo_code):
    # Policy will be used to code this function

    # Get the loyalty points for the customer
    loyalty_points_balance = get_customer_loyalty_points_balance.invoke({"customer_id": customer_id})

    # Get the loyalty discount based on loyalty points
    loyalty_points_discount_pct = get_discount_pct_for_loyalty.invoke({"loyalty_points_balance": loyalty_points_balance})

    # Get the promo discount based on promo code
    promo_code_discount_pct = get_discount_pct_for_promotion_code({"promo_code": promo_code})

    # Final discount
    final_discount = loyalty_points_discount_pct + promo_code_discount_pct

    # Apply max discount policy
    was_max_discount_policy_applied=False
    if final_discount > 0.25:
        was_max_discount_policy_applied = True
        final_discount = 0.25

    return {"customer_discount": final_discount, 
             "loyalty_points_balance": loyalty_points_balance,
            "loyalty_discount": loyalty_points_discount_pct, 
            "promo_discount": promo_code_discount_pct,
            "was_final_discount_adjusted": was_max_discount_policy_applied}

#### Single test

In [13]:
customer_id = 100
promo_code  = "summer2024"

response_from_harness = calculate_discount_without_LLM(customer_id, promo_code)
print("Result from test harness = ", json.dumps(response_from_harness, indent=4))

response_from_agent = agent_executor.invoke({"customer_id": customer_id, "promo_code": promo_code})

print("Response from agent:",response_from_agent['output'])

C:\Users\raj\anaconda3\envs\genai-course\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Result from test harness =  {
    "customer_discount": 0.15,
    "loyalty_points_balance": 342,
    "loyalty_discount": 0.0,
    "promo_discount": 0.15,
    "was_final_discount_adjusted": false
}
Response from agent: {
  "customer_discount": 0.15,
  "loyalty_points_balance": 342,
  "loyalty_discount": 0.0,
  "promo_discount": 0.15,
  "was_final_discount_adjusted": false
}


#### Multiple tests for accuracy

In [14]:
correct_count = 0
incorrect_count = 0
test_cases = [
    {"customer_id":100, "promo_code":"summer2024"},
    {"customer_id":200, "promo_code":"summer2024"},
    {"customer_id":300, "promo_code":"friends&family"},
    {"customer_id":400, "promo_code":"bad-code"},
    {"customer_id":500, "promo_code":"summer2024"},
    {"customer_id":100, "promo_code":"friends"},
    {"customer_id":300, "promo_code":"friends&family"},
]

for test_case in test_cases:
    print("----------------",test_case,"------------------")
    response_from_harness = calculate_discount_without_LLM(**test_case)
    response_from_agent = agent_executor.invoke(test_case)['output']

    # Apply the hack 
    response_from_agent = openai_chat_output_hack(response_from_agent)
    
    if response_from_harness['customer_discount'] == json.loads(response_from_agent)['customer_discount']:
        correct_count = correct_count + 1
        print('Correct')
    else:
        incorrect_count = incorrect_count + 1
        print('Incorrect')

---------------- {'customer_id': 100, 'promo_code': 'summer2024'} ------------------
Incorrect
---------------- {'customer_id': 200, 'promo_code': 'summer2024'} ------------------
Incorrect
---------------- {'customer_id': 300, 'promo_code': 'friends&family'} ------------------
Correct
---------------- {'customer_id': 400, 'promo_code': 'bad-code'} ------------------
Incorrect
---------------- {'customer_id': 500, 'promo_code': 'summer2024'} ------------------
Correct
---------------- {'customer_id': 100, 'promo_code': 'friends'} ------------------
Correct
---------------- {'customer_id': 300, 'promo_code': 'friends&family'} ------------------
Correct


In [15]:
accuracy = round(correct_count/(correct_count + incorrect_count),2)
print("Total: ",(correct_count+incorrect_count)," Correct: ", correct_count, "  Incorrect: ", incorrect_count, "  Accuracy: ", accuracy)

Total:  7  Correct:  4   Incorrect:  3   Accuracy:  0.57


In [16]:
response_from_agent

'{\n  "customer_discount": 0.2,\n  "loyalty_points_balance": 1001,\n  "loyalty_discount": 0.1,\n  "promo_discount": 0.1,\n  "was_final_discount_adjusted": false\n}'